In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import OrderedDict

In [27]:
from datetime import datetime

In [7]:
raw_df = pd.read_csv('./data/twitter_normal_user_data.csv', encoding = "ISO-8859-1")
raw_df

,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,description,fav_number,link_color,name,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_location,user_timezone
0,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,i sing my own rhythm.,0,08C2C2,sheezy0,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,main; @Kan1shk3,Chennai
1,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,I'm the author of novels filled with family dr...,68,0084B4,DavdBurnett,0,C0DEED,ÂÃÃIt felt like they were my friends and I ...,NaN,7471,10/26/15 12:40,NaN,Eastern Time (US & Canada)
2,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,louis whining and squealing and all,7696,ABB8C2,lwtprettylaugh,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,clcncl,Belgrade
3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",202,0084B4,douggarland,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,"Palo Alto, CA",Pacific Time (US & Canada)
4,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,37318,3B94D9,WilfordGemma,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,10/26/15 23:04,male,1.0000,yes,1.0,1/24/13 11:55,XXI,33834,ABB8C2,AntiHero94,1,0,Seriously love Trent and Justin. Would do anyt...,NaN,29190,10/26/15 13:18,"Nashville, TN",Central Time (US & Canada)
19996,10/27/15 1:40,male,1.0000,yes,1.0,12/19/14 13:54,607. Money in the making. Take it a day at a t...,2462,0084B4,dredoo24,0,C0DEED,One person can look at you like your the most ...,NaN,11117,10/26/15 13:18,NaN,NaN
19997,10/27/15 0:03,female,1.0000,yes,1.0,4/15/15 19:15,I can quote (Insert movie) better than you and...,1855,664422,6BETHANY4Victor,0,0,These 24 Wedding Photobombs Made The Bride And...,NaN,5640,10/26/15 13:18,Glendale CA,NaN
19998,10/26/15 22:27,female,0.6593,yes,1.0,4/6/15 13:43,| Kingdom Hearts | Alice | Sims | Minecraft | ...,2160,05B8CC,SorasNerdyGeek,0,0,So I am riding around and I saw a horse runnin...,NaN,2041,10/26/15 13:18,NaN,Eastern Time (US & Canada)


# Scheme 1:

In [52]:
df1 = raw_df.copy()

In [53]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _last_judgment_at      20000 non-null  object 
 1   gender                 19903 non-null  object 
 2   gender:confidence      19974 non-null  float64
 3   profile_yn             20000 non-null  object 
 4   profile_yn:confidence  20000 non-null  float64
 5   created                20000 non-null  object 
 6   description            16262 non-null  object 
 7   fav_number             20000 non-null  int64  
 8   link_color             20000 non-null  object 
 9   name                   20000 non-null  object 
 10  retweet_count          20000 non-null  int64  
 11  sidebar_color          20000 non-null  object 
 12  text                   20000 non-null  object 
 13  tweet_coord            159 non-null    object 
 14  tweet_count            20000 non-null  int64  
 15  tw

- If `gender` is not recorded, fill the value with unknown
- In such case (`gender` not recorded), `gender:confidence` is 0
- Mark NULL values

In [55]:
df1['gender:confidence'].fillna(0, inplace=True)

In [56]:
df1['gender'] = df1['gender'].apply(lambda x: 'unknown' if pd.isnull(x) else x)
df1['description'] = df1['description'].apply(lambda x: '' if pd.isnull(x) else x)
df1['tweet_coord'] = df1['tweet_coord'].apply(lambda x: '' if pd.isnull(x) else x)
df1['tweet_location'] = df1['tweet_location'].apply(lambda x: '' if pd.isnull(x) else x)
df1['user_timezone'] = df1['user_timezone'].apply(lambda x: '' if pd.isnull(x) else x)

In [57]:
def split_date(ddata):
    dlist = ddata.split()
    date_data = dlist[0]
    time_data = dlist[1]

    date_data_list = list(date_data.split('/'))
    date_data_list[2] = str(int(date_data_list[2]) + 2000)

    date_data_list.extend(list(time_data.split(':')))
    date_data_list.append("00")
    
    return ' '.join(date_data_list)


In [58]:
df1['_last_judgment_at'] = df1['_last_judgment_at'].apply(split_date)
df1['created'] = df1['created'].apply(split_date)
df1['tweet_created'] = df1['tweet_created'].apply(split_date)

In [59]:
df1.head()

,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,description,fav_number,link_color,name,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_location,user_timezone
0,10 26 2015 23 24 00,male,1.0000,yes,1.0,12 5 2013 1 48 00,i sing my own rhythm.,0,08C2C2,sheezy0,0,FFFFFF,Robbie E Responds To Critics After Win Against...,,110964,10 26 2015 12 40 00,main; @Kan1shk3,Chennai
1,10 26 2015 23 30 00,male,1.0000,yes,1.0,10 1 2012 13 51 00,I'm the author of novels filled with family dr...,68,0084B4,DavdBurnett,0,C0DEED,ÂÃÃIt felt like they were my friends and I ...,,7471,10 26 2015 12 40 00,,Eastern Time (US & Canada)
2,10 26 2015 23 33 00,male,0.6625,yes,1.0,11 28 2014 11 30 00,louis whining and squealing and all,7696,ABB8C2,lwtprettylaugh,1,C0DEED,i absolutely adore when louis starts the songs...,,5617,10 26 2015 12 40 00,clcncl,Belgrade
3,10 26 2015 23 10 00,male,1.0000,yes,1.0,6 11 2009 22 39 00,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",202,0084B4,douggarland,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,,1693,10 26 2015 12 40 00,"Palo Alto, CA",Pacific Time (US & Canada)
4,10 27 2015 1 15 00,female,1.0000,yes,1.0,4 16 2014 13 23 00,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,37318,3B94D9,WilfordGemma,0,0,Watching Neighbours on Sky+ catching up with t...,,31462,10 26 2015 12 40 00,,
